In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

c:\Users\Camille\Documents\TWR


In [1]:
from training_models.config.database import AsyncSessionLocal
from training_models.repositories.campaigns_repository import CampaignRepository
from training_models.repositories.request_repository import RequestsRepository
from training_models.config.mongo import mongo_collection, mongo_client

campaign_repository = CampaignRepository(session_factory=AsyncSessionLocal)
mongo_request_repository = RequestsRepository(collection=mongo_collection)

Garantindo índices...


c:\Users\Camille\Documents\TWR\mil\.venv\Lib\site-packages\motor\core.py:171: UserWarning: You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb
  delegate = self.__delegate_class__(*args, **kwargs)


In [2]:
from attention_based import MILTrainingService

c:\Users\Camille\Documents\TWR\mil\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using 11 out of 12 cores


In [ ]:
import torch

traffic_source_alvo = "google" # Altere para a fonte real

training_service = MILTrainingService(
    traffic_source=traffic_source_alvo,
    repo_requests=mongo_request_repository,
    repo_campaigns=campaign_repository,
    device="cuda" if torch.cuda.is_available() else "cpu",
    emb_config="fasttext",
    bag_size=3
)

# 4. Execução Direta (A mágica do Jupyter)
try:
    print(f"Iniciando pipeline para a fonte: {traffic_source_alvo}")
    
    # É só usar o await direto na célula!
    await training_service.training_pipeline(
        epochs=15, 
        path="models"
    )
    
except Exception as e:
    print(f"Ocorreu um erro durante o pipeline: {e}")
    
finally:
    # Fecha as conexões com o banco de dados
    print("Limpando conexões com o banco de dados...")


2026-02-27 11:56:36,002 - INFO - Inicializando MILTrainingService...
Fastext model path:  G:/Meu Drive/TWR/data/google/fasttext_google.model
[DEBUG] Model Path FASTTEXT: G:/Meu Drive/TWR/data/google/fasttext_google.model
2026-02-27 11:56:36,025 - INFO - loading FastText object from G:/Meu Drive/TWR/data/google/fasttext_google.model
2026-02-27 11:56:36,118 - INFO - loading wv recursively from G:/Meu Drive/TWR/data/google/fasttext_google.model.wv.* with mmap=None
2026-02-27 11:56:36,119 - INFO - loading vectors_vocab from G:/Meu Drive/TWR/data/google/fasttext_google.model.wv.vectors_vocab.npy with mmap=None
2026-02-27 11:56:36,419 - INFO - loading vectors_ngrams from G:/Meu Drive/TWR/data/google/fasttext_google.model.wv.vectors_ngrams.npy with mmap=None
2026-02-27 11:56:53,231 - INFO - setting ignored attribute vectors to None
2026-02-27 11:56:53,242 - INFO - setting ignored attribute buckets_word to None


In [ ]:
# hashes = ["0h5ff0hx9e", "3zdba5h0e9", "9ct9zcf7xe", 'eoqs2p9hvl', "gp0cj2b193", "mdwcdjlo0h"]

In [ ]:
hashes = await campaign_repository.get_recent_active_campaign_hashes(limit=10, traffic_source=traffic_source_alvo)

In [ ]:
import pandas as pd

results, hashes_info = await mongo_request_repository.get_training_sample_by_hashes(
      hashes=hashes, 
      limit_each=500,
      rule_id=True
)
df = pd.DataFrame(results)

[{'_id': '962x48y418', 'start_datetime': datetime.datetime(2026, 2, 26, 14, 3, 5, 785000), 'end_datetime': datetime.datetime(2026, 2, 27, 2, 40, 12, 690000)}, {'_id': '3lss196h64', 'start_datetime': datetime.datetime(2026, 2, 27, 8, 4, 16, 745000), 'end_datetime': datetime.datetime(2026, 2, 27, 8, 22, 17, 943000)}, {'_id': 'w4ie3dolyo', 'start_datetime': datetime.datetime(2026, 2, 27, 12, 50, 2, 196000), 'end_datetime': datetime.datetime(2026, 2, 27, 13, 46, 31, 555000)}, {'_id': '87b31f7d9c', 'start_datetime': datetime.datetime(2026, 2, 26, 15, 18, 30, 823000), 'end_datetime': datetime.datetime(2026, 2, 27, 1, 35, 56, 589000)}, {'_id': 'g35z3inn2l', 'start_datetime': datetime.datetime(2026, 2, 27, 6, 25, 56, 821000), 'end_datetime': datetime.datetime(2026, 2, 27, 8, 8, 56, 478000)}, {'_id': '8o85jcb6lp', 'start_datetime': datetime.datetime(2026, 2, 27, 12, 54, 27, 188000), 'end_datetime': datetime.datetime(2026, 2, 27, 14, 7, 54, 787000)}, {'_id': 'c2c4ajjwxj', 'start_datetime': datet

In [ ]:
import torch

from attention_based import AttentionMIL

modelo = AttentionMIL(in_features=300, hidden_dim=256)
checkpoint = torch.load(f"models/{traffic_source_alvo}/fasttext/attention_mil.pth", weights_only=False)
pesos = checkpoint["model_state_dict"]

modelo.load_state_dict(pesos)

modelo.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import ipaddress

def extract_ip_stack(ip_string):
      try:
            ip = ipaddress.ip_address(ip_string)
            if ip.version == 4:
                  return str(ipaddress.ip_network(f"{ip_string}/24", strict=False))
            elif ip.version == 6:
                  return str(ipaddress.ip_network(f"{ip_string}/48", strict=False))
      except ValueError:
            return "ip_invalido"

In [ ]:
from asyncio.log import logger
from attention_based import MILBagDatasetLogical
from torch.utils.data import DataLoader
from data.embedding_service import EmbeddingService

df["decision"] = df["decision"].str.lower().replace({"bot": "bots"})

mapeamento_mil = {"bots": 1, "unsafe": 0}
df["decision_mil"] = df["decision"].map(mapeamento_mil)

PATH = "G:/Meu Drive/TWR/data"
LABEL_MAP = {"bot": 0, "unsafe": 1, "bots": 0}
conf = "fasttext"
model_path_ft = f"{PATH}/{traffic_source_alvo}/fasttext_{traffic_source_alvo}.model"  
# model_path_ft = "all-MiniLM-L6-v2"

try:
      EmbeddingService.get_instance(conf, model_path_ft, traffic_source=traffic_source_alvo)
except FileNotFoundError:
      print(f"Error: Embedding model not found at {model_path_ft}")

# df["ip_block"] = df["ip"].apply(extract_ip_stack)
# df["ip_api_isp"] = df["ip_api_isp"].fillna("ip_unknow")
# #3 bag id -> utilizando pelo MIL Dataset para agrupar
# df["bag_id"] = df["ip_block"] + " | " + df["ip_api_isp"]

embeddings_matrix, texts = EmbeddingService.process_and_encode(df)
df["embedding"] = list(embeddings_matrix)

logger.info("Agrupando requisições em Bags por Endereço IP...")
bags_df = df.groupby("ip").agg({
      "embedding": list,
      "decision_mil": list,
      # "ip": list
}).reset_index()

bags_df["bag_label"] = bags_df["decision_mil"].apply(lambda labels: 1.0 if 1 in labels else 0.0)

logger.info(f"Total de IPs (Bags) formados: {len(bags_df)}")

dataset_test = MILBagDatasetLogical(bags_df)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

Enter to Fasttext encoder


Criando Vocabulário: 100%|██████████| 50/50 [00:00<00:00, 1879.88it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 50/50 [00:00<00:00, 72.30it/s] 

Finishing encoding
2026-02-27 11:15:49,307 - INFO - Agrupando requisições em Bags por Endereço IP...
2026-02-27 11:15:49,319 - INFO - Total de IPs (Bags) formados: 43


In [ ]:
import json
import numpy as np

def cacar_bots_camuflados(modelo, dataloader, df_original, device, threshold=0.5, limite_exibicao=5):
    """
    Procura IPs que estão rotulados como 'unsafe' (0.0) no banco de dados,
    mas que o algoritmo MIL classificou como 'bot' (1.0) com alta certeza.
    """
    modelo.eval()
    bots_camuflados_encontrados = 0
    
    print("🕵️‍♂️ Iniciando caçada aos Bots Camuflados...")
    print(f"Buscando IPs rotulados como Humanos, mas previstos como Bot (> {threshold*100}%)\n")
    print("=" * 60)
    
    with torch.no_grad():
        for bag, label, bag_id in dataloader:
            
            # 1. Filtro do Banco de Dados: Só queremos olhar para quem foi rotulado como Humano (0.0)
            if label.item() != 0.0:
                continue
                
            bag = bag.to(device)
            pred, attention = modelo(bag)
            
            # 2. Filtro do Modelo: O modelo discorda do banco e tem certeza de que é um Bot?
            if pred.item() > threshold:
                bag_string = bag_id[0]
                bots_camuflados_encontrados += 1
                
                print(f"🚨 BOT CAMUFLADO DETECTADO | IP: {bag_string}")
                print(f"   Certeza da IA: {pred.item() * 100:.2f}%")
                
                # 3. Busca o Histórico no DataFrame Original
                # Pega todas as linhas originais desse IP
                logs_do_ip = df_original[df_original['ip'] == bag_string].reset_index(drop=True)
                print(len(logs_do_ip))
                
                # Extrai os pesos de atenção para sabermos qual requisição entregou o disfarce
                pesos = attention.squeeze().cpu().numpy()
                if pesos.ndim == 0:
                    pesos = np.array([pesos])
                
                # Encontra o índice da requisição que recebeu a maior nota de culpa
                idx_maior_peso = np.argmax(pesos)
                
                print(f"   Total de Requisições na Sessão: {len(logs_do_ip)}")
                print(f"   Requisição que desmascarou o Bot (Peso: {pesos[idx_maior_peso]:.4f}):")
                
                # 4. Imprime o Texto Original (Request/Headers)
                linha_culpada = logs_do_ip.iloc[idx_maior_peso]
                
                # Exibe a URL acessada e os Headers
                print(f"   -> Headers: ")
                json_header = json.loads(linha_culpada.get('headers', 'Coluna request não encontrada'))
                print(json.dumps(json_header, indent=4))
                # Se você tiver a coluna headers como string/dict, podemos imprimi-la também:
                # print(f"   -> Headers: {linha_culpada.get('headers', 'N/A')}")
                
                print("-" * 60)
                
                if bots_camuflados_encontrados >= limite_exibicao:
                    print(f"Parando a exibição nos primeiros {limite_exibicao} bots encontrados.")
                    break
                    
    if bots_camuflados_encontrados == 0:
        print("Nenhum bot camuflado encontrado neste lote de teste! O seu banco de dados está bem limpo.")

In [ ]:
# # Chama a função para inspecionar os 10 primeiros bots camuflados que o modelo encontrar
# cacar_bots_camuflados(
#     modelo=modelo, 
#     dataloader=test_loader, 
#     df_original=df, 
#     device=device, 
#     threshold=0.5, 
#     limite_exibicao=10
# )

## Testando com ids falsos

In [ ]:
# df["rule_id_list"].value_counts()

In [ ]:
# idx_com_1170 = df[
#     df["rule_id_list"].apply(lambda x: isinstance(x, list) and 1169 in x)
# ].index

# percentual = 0.3

# idx_sorteados = idx_com_1170.to_series().sample(
#     frac=percentual,
#     random_state=42 
# ).index

In [ ]:
# df.loc[idx_sorteados, "decision"] = "unsafe"
# df.loc[idx_sorteados, ["decision", "rule_id_list"]].head()

In [ ]:
# import numpy as np

# df["changed"] = False
# df.loc[idx_sorteados, "changed"] = True
# df["changed"].value_counts()

In [ ]:
# bags_df_test = df.groupby("ip").agg({
#     "embedding": list,
#     "decision_mil": list,
#     "changed": list,
#     "headers": list
# }).reset_index()

# bags_df_test["tem_isca"] = bags_df_test["changed"].apply(lambda lista: True in lista)

# print(f"Total de IPs agrupados: {len(bags_df_test)}")
# print(f"Total de IPs com iscas (modificados por você): {bags_df_test['tem_isca'].sum()}")

In [ ]:
bags_df_test = df.groupby("ip").agg({
    "embedding": list,
    "decision_mil": list,
    "headers": list
}).reset_index()

In [ ]:
filtro_mistas = bags_df_test["decision_mil"].apply(lambda lista: (1 in lista) and (0 in lista))
bags_mistas = bags_df_test[filtro_mistas]
display(bags_mistas[["ip", "decision_mil", "headers"]])

,ip,decision_mil,headers


In [ ]:
# analise = bags_mistas.iloc[0]["headers"]

# json_analise = json.loads(analise[1])
# # print(json.dumps(json_analise, indent=4))

In [ ]:
# json_analise = json.loads(analise[0])
# # print(json.dumps(json_analise, indent=4))

In [ ]:
# import torch
# import numpy as np

# def auditar_injecao_adversarial(modelo, dataloader, bags_df_test, device):
#     modelo.eval()
#     acertos_bag = 0
#     acertos_atencao = 0
#     total_iscas = 0
    
#     print("🎯 Iniciando Auditoria de Teste Adversarial...")
#     print("Verificando se o modelo encontra as suas modificações...\n" + "="*60)
    
#     with torch.no_grad():
#         # 1. Mudança: O terceiro elemento agora é a tupla do Bag ID
#         for bag, label, bag_id_tuple in dataloader:
#             # O DataLoader coloca a string dentro de uma tupla/lista por causa do batch
#             bag_id_string = bag_id_tuple[0] 
            
#             # 2. Mudança: Filtramos o DataFrame usando a nova coluna 'bag_id'
#             filtro_df = bags_df_test[bags_df_test['ip'] == bag_id_string]
            
#             # Proteção caso o bag_id não seja encontrado (evita erro de índice fora dos limites)
#             if filtro_df.empty:
#                 continue
                
#             info_bag = filtro_df.iloc[0]
            
#             if not info_bag.get('tem_isca', False):
#                 continue
                
#             total_iscas += 1
#             lista_changed = info_bag['changed'] 
            
#             bag = bag.to(device)
#             pred, attention = modelo(bag)
#             certeza = pred.item()
            
#             # 3. Mudança: Atualizamos o log para refletir a nova granularidade
#             print(f"\n🕵️ Investigando Bag ID Injetado: {bag_id_string} | Total de Reqs na Sessão: {len(lista_changed)}")

#             if certeza > 0.5:
#                 print(f"  [1. BAG] ✅ SUCESSO: Modelo cravou como BOT com {certeza*100:.2f}% de certeza.")
#                 acertos_bag += 1
#             else:
#                 print(f"  [1. BAG] ❌ FALHA: O modelo achou que era humano ({certeza*100:.2f}%). O seu disfarce foi perfeito.")

#             pesos = attention.squeeze().cpu().numpy()
#             if pesos.ndim == 0: pesos = np.array([pesos])
            
#             idx_maior_peso = np.argmax(pesos)
            
#             req_estava_modificada = lista_changed[idx_maior_peso]
            
#             if req_estava_modificada:
#                 print(f"  [2. ATENÇÃO] 🎯 SUCESSO: O modelo apontou exatamente para o índice {idx_maior_peso}, que foi o que você adulterou!")
#                 acertos_atencao += 1
#             else:
#                 print(f"  [2. ATENÇÃO] ⚠️ AVISO: O modelo focou no índice {idx_maior_peso} (Peso: {pesos[idx_maior_peso]:.2f}), mas você tinha alterado outro índice. Ele se confundiu ou achou outro bot real.")
                
#     print("\n" + "="*60)
#     print("📊 RESUMO FINAL DA AUDITORIA:")
#     print(f"Total de Bags Adulteradas Testadas: {total_iscas}")
#     if total_iscas > 0:
#         print(f"Taxa de Captura da Bag (Bloco IP + ISP): {acertos_bag/total_iscas * 100:.1f}%")
#         print(f"Taxa de Precisão da Atenção (Na Mosca): {acertos_atencao/total_iscas * 100:.1f}%")

# # Chamada da função:
# auditar_injecao_adversarial(modelo, test_loader, bags_df_test, device)

In [ ]:
resultados_finais = []
with torch.no_grad():
      for _, row in bags_df.iterrows():
            ip_atual = row["ip"]
            bag_tensor = torch.tensor(row["embedding"], dtype=torch.float32).unsqueeze(0).to(device)
            
            pred, attention = modelo(bag_tensor)
            certeza_bag = pred.item()
            classe_predita = 1 if certeza_bag > 0.5 else 0
            
            pesos = attention.squeeze().cpu().numpy()
            if pesos.ndim == 0: 
                  pesos = [pesos.item()]
            else: 
                  pesos = pesos.tolist()

            linhas_do_ip = df[df["ip"] == ip_atual].copy()
            linhas_do_ip["pred"] = classe_predita
            linhas_do_ip["certeza_bag"] = round(certeza_bag * 100, 2)
            
            linhas_do_ip["attention_weight"] = [pesos[i] if i < len(pesos) else 0.0 for i in range(len(linhas_do_ip))]
            
            resultados_finais.append(linhas_do_ip)
      
df_completo = pd.concat(resultados_finais, ignore_index=True)

In [ ]:
df_completo["target"] = df_completo["decision"].map(mapeamento_mil) 
df_completo['is_error'] = df_completo["target"] != df_completo["pred"]

# Conta quantos valores True existem na coluna
erros = df_completo['is_error'].sum() 
erros

np.int64(2)

In [ ]:
df_completo["pred"].value_counts()

pred
0    27
1    23
Name: count, dtype: int64

In [ ]:
df_completo[df_completo["is_error"] == True]

,_id,datetime,decision,headers,ip,ip_api_isp,request,decision_mil,embedding,pred,certeza_bag,attention_weight,target,is_error
11,69a1a4d3b61c8b614a8a78a3,2026-02-27 14:06:11.937,bots,"{""Host"":""aff.conectadosimples.online"",""X-Reque...",201.77.112.132,desktop sigmanet comunicação multimídia sa,"{""cr"":""0"",""plc"":"""",""mtx"":""b"",""rdn"":""1234567890...",1,"[0.116934925, 0.5183307, -0.12576239, 0.145394...",0,17.90,1.0,1,True
26,69a1a4d7b61c8b614a8a7b36,2026-02-27 14:06:15.405,bots,"{""Host"":""aff.conectadosimples.online"",""X-Reque...",45.185.25.36,itminds consultoria em tecnologia da informacao,"{""cr"":""0"",""plc"":"""",""mtx"":""b"",""rdn"":""1234567890...",1,"[0.10898504, 0.5258511, -0.13055888, 0.1607674...",0,31.11,1.0,1,True


In [ ]:
import os
import torch
import pandas as pd
from asyncio.log import logger
from torch.utils.data import DataLoader

# Importe suas classes locais
from attention_based import AttentionMIL, MILBagDatasetLogical
from data.embedding_service import EmbeddingService

# 1. Defina a sua lista de fontes e os caminhos base
lista_traffic_sources = ["google", "facebook", "tiktok", "taboola"] # Adicione as fontes que desejar
PATH_EMB = "G:/Meu Drive/TWR/data"
PATH_MODELS = "models"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Inicia o Loop de Automação
for traffic_source in lista_traffic_sources:
    print(f"\n{'='*60}\n🚀 INICIANDO PIPELINE PARA A FONTE: {traffic_source.upper()}\n{'='*60}")
    
    model_path_mil = f"{PATH_MODELS}/{traffic_source}/fasttext/attention_mil.pth"
    
    # ---------------------------------------------------------
    # ETAPA A: TREINAMENTO OU CARREGAMENTO
    # ---------------------------------------------------------
    if os.path.exists(model_path_mil):
        print(f"✅ Modelo já treinado encontrado em: {model_path_mil}\n⏭️ Pulando etapa de treinamento...")
    else:
        print(f"⚙️ Modelo não encontrado. Iniciando Treinamento do zero...")
        try:
            training_service = MILTrainingService(
                traffic_source=traffic_source,
                repo_requests=mongo_request_repository,
                repo_campaigns=campaign_repository,
                device=DEVICE.type,
                emb_config="fasttext",
                bag_size=3
            )
            # Treina e salva o modelo
            await training_service.training_pipeline(epochs=15, path=PATH_MODELS)
            print("✅ Treinamento concluído com sucesso!")
        except Exception as e:
            print(f"❌ Erro crítico no treinamento de {traffic_source}: {e}")
            continue # Pula para a próxima fonte se o treino quebrar

    # ---------------------------------------------------------
    # ETAPA B: INGESTÃO DE DADOS DE TESTE
    # ---------------------------------------------------------
    print(f"\n🔍 Extraindo dados de teste do banco...")
    try:
        hashes = await campaign_repository.get_recent_active_campaign_hashes(limit=20, traffic_source=traffic_source)
        results, hashes_info = await mongo_request_repository.get_training_sample_by_hashes(
            hashes=hashes, 
            limit_each=500,
            rule_id=True
        )
        
        if not results:
            print(f"⚠️ Nenhum dado de teste encontrado para {traffic_source}. Pulando avaliação.")
            continue
            
        df = pd.DataFrame(results)
        
        # Tratamento do Gabarito (Label)
        df["decision"] = df["decision"].str.lower().replace({"bot": "bots"})
        df["decision_mil"] = df["decision"].map({"bots": 1, "unsafe": 0})
        
        # Previne erros limpando possíveis linhas sem classificação
        df = df.dropna(subset=["decision_mil"]).copy() 
        
    except Exception as e:
         print(f"❌ Erro ao buscar dados de teste para {traffic_source}: {e}")
         continue

    # ---------------------------------------------------------
    # ETAPA C: GERAÇÃO DE EMBEDDINGS E AGRUPAMENTO
    # ---------------------------------------------------------
    print("🧠 Processando Embeddings e agrupando Bags...")
    model_path_ft = f"{PATH_EMB}/{traffic_source}/fasttext_{traffic_source}.model"
    
    try:
        EmbeddingService.get_instance("fasttext", model_path_ft, traffic_source=traffic_source)
        embeddings_matrix, _ = EmbeddingService.process_and_encode(df)
        df["embedding"] = list(embeddings_matrix)
    except FileNotFoundError:
        print(f"❌ Erro: Modelo FastText não encontrado em {model_path_ft}. Pulando fonte.")
        continue

    bags_df = df.groupby("ip").agg({"embedding": list}).reset_index()

    # ---------------------------------------------------------
    # ETAPA D: INFERÊNCIA MIL E DESEMPACOTAMENTO
    # ---------------------------------------------------------
    print("⚖️ Executando Inferência Attention MIL...")
    modelo = AttentionMIL(in_features=300, hidden_dim=256)
    checkpoint = torch.load(model_path_mil, weights_only=False)
    modelo.load_state_dict(checkpoint["model_state_dict"])
    modelo.eval().to(DEVICE)

    resultados_finais = []
    with torch.no_grad():
        for _, row in bags_df.iterrows():
            ip_atual = row["ip"]
            bag_tensor = torch.tensor(row["embedding"], dtype=torch.float32).unsqueeze(0).to(DEVICE)
            
            pred, attention = modelo(bag_tensor)
            certeza_bag = pred.item()
            classe_predita = 1 if certeza_bag > 0.5 else 0
            
            pesos = attention.squeeze().cpu().numpy()
            if pesos.ndim == 0: pesos = [pesos.item()]
            else: pesos = pesos.tolist()

            linhas_do_ip = df[df["ip"] == ip_atual].copy()
            linhas_do_ip["pred"] = classe_predita
            linhas_do_ip["certeza_bag"] = round(certeza_bag * 100, 2)
            linhas_do_ip["attention_weight"] = [pesos[i] if i < len(pesos) else 0.0 for i in range(len(linhas_do_ip))]
            
            resultados_finais.append(linhas_do_ip)

    df_completo = pd.concat(resultados_finais, ignore_index=True)

    # ---------------------------------------------------------
    # ETAPA E: AVALIAÇÃO DE DESEMPENHO (MÉTRICAS)
    # ---------------------------------------------------------
    acuracia = (df_completo["decision_mil"] == df_completo["pred"]).mean()
    total_erros = (df_completo["decision_mil"] != df_completo["pred"]).sum()
    
    fp = len(df_completo[(df_completo["decision_mil"] == 0) & (df_completo["pred"] == 1)])
    fn = len(df_completo[(df_completo["decision_mil"] == 1) & (df_completo["pred"] == 0)])

    print("\n📊 RELATÓRIO DE AVALIAÇÃO DO MODELO:")
    print(f"- Total de Linhas Analisadas: {len(df_completo)}")
    print(f"- Acurácia Global: {acuracia * 100:.2f}%")
    print(f"- Total de Discrepâncias: {total_erros}")
    print(f"- Falsos Positivos (Humano julgado como Bot): {fp}")
    print(f"- Falsos Negativos (Bot julgado como Humano): {fn}")
    print(f"- Distribuição das Previsões do Modelo:\n{df_completo['pred'].value_counts().to_string()}")

print(f"\n{'='*60}\n🏁 PIPELINE EM LOTE FINALIZADO!\n{'='*60}")


🚀 INICIANDO PIPELINE PARA A FONTE: GOOGLE
✅ Modelo já treinado encontrado em: models/google/fasttext/attention_mil.pth
⏭️ Pulando etapa de treinamento...

🔍 Extraindo dados de teste do banco...
[{'_id': 'w4ie3dolyo', 'start_datetime': datetime.datetime(2026, 2, 27, 12, 50, 2, 196000), 'end_datetime': datetime.datetime(2026, 2, 27, 14, 34, 26, 575000)}, {'_id': 'vmj7kpsoi1', 'start_datetime': datetime.datetime(2026, 2, 26, 11, 39, 50, 318000), 'end_datetime': datetime.datetime(2026, 2, 27, 15, 19, 2, 506000)}, {'_id': 'g35z3inn2l', 'start_datetime': datetime.datetime(2026, 2, 27, 6, 25, 56, 821000), 'end_datetime': datetime.datetime(2026, 2, 27, 8, 8, 56, 478000)}, {'_id': 'q2wq4mc73u', 'start_datetime': datetime.datetime(2026, 2, 26, 13, 48, 31, 381000), 'end_datetime': datetime.datetime(2026, 2, 27, 15, 44, 57, 492000)}, {'_id': 'c2c4ajjwxj', 'start_datetime': datetime.datetime(2026, 2, 27, 12, 51, 21, 490000), 'end_datetime': datetime.datetime(2026, 2, 27, 13, 40, 3, 75000)}, {'_id'

Criando Vocabulário: 100%|██████████| 1000/1000 [00:00<00:00, 2187.99it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 1000/1000 [00:09<00:00, 107.86it/s]


Finishing encoding
⚖️ Executando Inferência Attention MIL...

📊 RELATÓRIO DE AVALIAÇÃO DO MODELO:
- Total de Linhas Analisadas: 1000
- Acurácia Global: 99.20%
- Total de Discrepâncias: 8
- Falsos Positivos (Humano julgado como Bot): 2
- Falsos Negativos (Bot julgado como Humano): 6
- Distribuição das Previsões do Modelo:
pred
0    504
1    496

🚀 INICIANDO PIPELINE PARA A FONTE: FACEBOOK
✅ Modelo já treinado encontrado em: models/facebook/fasttext/attention_mil.pth
⏭️ Pulando etapa de treinamento...

🔍 Extraindo dados de teste do banco...
[{'_id': 'ri8r07x0b0', 'start_datetime': datetime.datetime(2026, 2, 27, 14, 38, 25, 675000), 'end_datetime': datetime.datetime(2026, 2, 27, 14, 41, 31, 24000)}, {'_id': 'dze6tr3h04', 'start_datetime': datetime.datetime(2026, 2, 27, 14, 42, 16, 390000), 'end_datetime': datetime.datetime(2026, 2, 27, 14, 45, 21, 343000)}, {'_id': '9t7x1urfpj', 'start_datetime': datetime.datetime(2026, 2, 27, 13, 22, 16, 406000), 'end_datetime': datetime.datetime(2026, 2

...

Criando Vocabulário: 100%|██████████| 1000/1000 [00:00<00:00, 1242.31it/s]


Using 11 out of 12 cores


Vetorizando: 100%|██████████| 1000/1000 [00:13<00:00, 76.34it/s]


Finishing encoding
⚖️ Executando Inferência Attention MIL...

📊 RELATÓRIO DE AVALIAÇÃO DO MODELO:
- Total de Linhas Analisadas: 1000
- Acurácia Global: 69.30%
- Total de Discrepâncias: 307
- Falsos Positivos (Humano julgado como Bot): 1
- Falsos Negativos (Bot julgado como Humano): 306
- Distribuição das Previsões do Modelo:
pred
0    805
1    195

🏁 PIPELINE EM LOTE FINALIZADO!


: 